Connect to elasticsearch

In [8]:
# Connection to Elasticsearch 
from pprint import pprint
from elasticsearch import Elasticsearch
from config.development import Config

es = Elasticsearch(
    Config.ES_HOST, 
    verify_certs=True, 
    basic_auth=(Config.ES_USERNAME, Config.ES_PASSWORD)
)

client_info = es.info()
print("Connected to the elasticsearch ✔")
pprint(client_info.body)

Connected to the elasticsearch ✔
{'cluster_name': 'elasticsearch',
 'cluster_uuid': 'tdYmEtALQyuf5oSJGl74OQ',
 'name': 'david-server',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-06-19T01:37:57.847711500Z',
             'build_flavor': 'default',
             'build_hash': '580aff1a0064ce4c93293aaab6fcc55e22c10d1c',
             'build_snapshot': False,
             'build_type': 'deb',
             'lucene_version': '8.11.3',
             'minimum_index_compatibility_version': '6.0.0-beta1',
             'minimum_wire_compatibility_version': '6.8.0',
             'number': '7.17.29'}}


In [9]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(index="my_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

Using a dummy_data.json file to insert multiple document in index. We stored the id of each document in document_id

In [10]:
import json
from tqdm import tqdm

document_ids = []
dummy_data = json.load(open("../data/dummy_data.json"))
for document in tqdm(dummy_data, total=len(dummy_data)):
    response = es.index(index="my_index", body=document)
    document_ids.append(response['_id'])

100%|██████████| 5/5 [00:00<00:00, 30.21it/s]


In [11]:
document_ids

['Le498ZkBtfwvgmKSH8H_',
 'Lu498ZkBtfwvgmKSIME6',
 'L-498ZkBtfwvgmKSIMFU',
 'MO498ZkBtfwvgmKSIMFx',
 'Me498ZkBtfwvgmKSIMGL']

Get Operation 
This is an example of successfull operation. If the ID exists in the document, to get operation wont return 
any error

In [12]:
response = es.get(index="my_index", id=document_ids[0])

In [13]:
from pprint import pprint

pprint(response.body)

{'_id': 'Le498ZkBtfwvgmKSH8H_',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_source': {'content': 'This is sample content first entry',
             'created_on': '2023-10-01',
             'title': 'Sample title 1'},
 '_type': '_doc',
 '_version': 1,
 'found': True}


In [14]:
# try: 
#     response = es.get(index="my_index", _id=3)
# except: 
#     print(e)